# UAV product workflow

In order for UAV data to be published in the DEPWS spatial library UAV data procedures require that the data has been processed using the approved methodology, and that the final product has been composited (where applicable) clipped, renamed with its Meta data updated.

This notebook facilities the processing of UAV images by:
 - Compositing grayscale images form the multispectral camera
 - Clipping images with an extent vector
 - Renaming the output in the required format
 - Updating final product metadata.


## Import modules

In [ ]:
import os
import shutil
import gdal
import numpy as np
from osgeo import gdal, gdalconst

In [ ]:
site = "wtr05"
dir_ = "mosaic_n"
directory = r"Z:\Scratch\uav\2022"

In [ ]:
os.chdir(r"{0}\{1}\{2}".format(directory, site, dir_))

In [ ]:
ls

## Composite greyscale images

In [ ]:
cd C:\Users\rmcgr\ntg_repo\uav\scripts

### Composite Multi band UAV images

In [ ]:
#%run run_composite_multi_bands_image.py -h

In [ ]:
%run run_composite_multi_bands_image.py \
--directory Z:\Scratch\uav\2022\wtr05\mosaic_n  \
    --outfile Z:\Scratch\uav\2022\wtr05\mosaic_n\p4ch_wtr05_20220409_recnm2.tif

## Clip raster with extent shapefile

In [ ]:
os.listdir(r"{0}\{1}\mosaic_n".format(directory, site))

In [ ]:
if not os.path.isdir(r"{0}\{1}\shp".format(directory, site)):   
    os.mkdir(r"{0}\{1}\shp".format(directory, site))
    
os.listdir(r"{0}\{1}\shp".format(directory, site))

## Multispectral Composite Clip

In [ ]:
cd C:\Users\rmcgr\ntg_repo\uav\scripts

In [ ]:
%run clip_tool_rasterio.py \
--raster Z:\Scratch\uav\2022\wtr05\mosaic_n\p4ch_wtr05_20220409_recnm2.tif \
    --shpfile Z:\Scratch\uav\2022\wtr05\shp\wtr05_extent.shp \
        --outfile Z:\Scratch\uav\2022\wtr05\p4ch_wtr05_20220409_refnm2.tif

## RGB Clip

In [ ]:
cd C:\Users\rmcgr\ntg_repo\uav\scripts

In [ ]:
%run clip_tool_rasterio.py -h

In [ ]:
%run clip_tool_rasterio.py \
--raster Z:\Scratch\uav\2022\wtr04\mosaic_n\p4gh_wtr04_n_mosaic_group1.tif \
    --shpfile Z:\Scratch\uav\2022\wtr04\shp\wtr04_extent.shp \
        --outfile Z:\Scratch\uav\2022\wtr04\p4gh_wtr04_20220408_rgbnm2.tif

## Metadata update

### All UAV final products

In [ ]:
cd C:\Users\rmcgr\ntg_repo\uav\scripts

In [ ]:
flight_date = '20220409'
pilot = 'Robert McGregor'
processing_date = "20220505"
utm = '52'
image_code = 'p4ch'

rgb = '' # y or n

In [ ]:
if rgb == 'y':
    image = r"{0}\{1}\{3}_{1}_{2}_rgbnm{4}.tif".format(directory, site, flight_date, image_code, utm[1])
else:
    image = r"{0}\{1}\{3}_{1}_{2}_refnm{4}.tif".format(directory, site, flight_date, image_code, utm[1])
    
print('Update metadata for: ',  image)
gdal.AllRegister()

inputdataset = gdal.Open(image)

inputdataset.SetMetadataItem("Flight","{0}".format(site))

inputdataset.SetMetadataItem("Flight date","{0}".format(flight_date))
inputdataset.SetMetadataItem("Pilot","{0}".format(pilot))

# platform and sensor
if image[-28:-27] == 'a':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","Micasense RedEdge-3")
    bands = "blue, green, red, red edge, nir"
elif image[-28:-27] == 'b':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","Micasence RedEdge-M") 
    bands = "blue, green, red, red edge, nir"
elif image[-28:-27] == 'c':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","DJI Multi WM331A")
    bands = "blue, green, red, red edge, nir"
elif image[-28:-27] == 'd':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","DJI RGB WM331A")  
    bands = "blue, green, red"
elif image[-28:-27] == 'e':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","DJI RGB WM331A") 
    bands = "blue, green, red"
elif image[-28:-27] == 'g':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","DJI RGB WM331A")
    bands = "blue, green, red"
else:
    pass
    
inputdataset.SetMetadataItem("Processing date","{0}".format(processing_date))
                             
#datum                             
if utm == '52':                     
    inputdataset.SetMetadataItem("Datum","WGS 1984")
    inputdataset.SetMetadataItem("Projection","UTM zone 52S")
    inputdataset.SetMetadataItem("EPSG","32752")
elif utm == '53':
    inputdataset.SetMetadataItem("Datum","WGS 1984")
    inputdataset.SetMetadataItem("Projection","UTM zone 53S")
    inputdataset.SetMetadataItem("EPSG","32753") 
else:
    pass

gcp = image[-7:-6]
if gcp == 'g':
    inputdataset.SetMetadataItem("Geo-rectified","Yes")
elif gcp == 'n':
    inputdataset.SetMetadataItem("Geo-rectified","No")
else:
    pass

inputdataset.SetMetadataItem("Band composition","{0}".format(bands))

inputdataset = None

src = gdal.Open(image, gdalconst.GA_ReadOnly)

print(src.GetMetadata())

In [ ]:
shutil.rmtree(r"{0}\{1}\mosaic_n".format(directory, site))

In [ ]:
shutil.rmtree(r"{0}\{1}\mosaic_g".format(directory, site))

In [ ]:
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio

In [ ]:
image1 = r"Z:\Scratch\uav\2022\gry03\mosaic_n1\p4ch_gry03_20220525_recnm2.tif"
image2 = r"Z:\Scratch\uav\2022\gry03\mosaic_n2\p4ch_gry03_20220525_recnm2_.tif"

In [ ]:
raster_files = [image1, image2]
raster_to_mosiac = []

In [ ]:
for p in raster_files:
    raster = rio.open(p)
    raster_to_mosiac.append(raster)

In [ ]:
mosaic, output = merge(raster_to_mosiac)

In [ ]:
output_meta = raster.meta.copy()
output_meta.update(
    {"driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": output,
    }
)

In [ ]:
output_path = r'Z:\Scratch\uav\2022\gry03\mosaic_output.tif'

In [ ]:
with rio.open(output_path, 'w', **output_meta) as m:
    m.write(mosaic)